<a href="https://colab.research.google.com/github/teamgaon/KIST/blob/main/220520_KIST_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ttach
!pip install albumentations==0.4.6

In [ ]:
!nvidia-smi
# k80 -> T4 -> P100

In [ ]:
import albumentations
import albumentations.pytorch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
import ttach as tta
import albumentations as A
from sklearn.model_selection import KFold, StratifiedKFold
from albumentations.pytorch import ToTensorV2
from datetime import datetime, timedelta
import random
import torchvision.transforms as transforms
import gc
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_cases = glob('/content/drive/MyDrive/KIST/open/train/*')
train_cases.sort()

In [ ]:
images = glob(train_cases[0]+'/image/*')
images.sort()
label = pd.read_csv(train_cases[0]+'/label.csv')
for i in range(len(images)):
  image = cv2.imread(images[i])
  print(images[i][-13:])
  print('leaf_weight : ' + str(label['leaf_weight'].loc[i]))
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# 분석에 사용할 feature 선택
csv_features = ['내부온도관측치', '외부온도관측치', '내부습도관측치', '외부습도관측치', 'CO2관측치', 'EC관측치',
       '최근분무량', '화이트 LED동작강도', '레드 LED동작강도', '블루 LED동작강도', '냉방온도', '냉방부하',
       '난방온도', '난방부하', '총추정광량', '백색광추정광량', '적색광추정광량']

csv_files = sorted(glob('/content/drive/MyDrive/KIST/open/train/*/meta/*'))

temp_csv = pd.read_csv(csv_files[0])[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_csv = temp_csv.fillna(0)
    if len(temp_csv) == 0:
        continue
    temp_csv = temp_csv.astype(float)
    temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
    max_arr = np.max([max_arr,temp_max], axis=0)
    min_arr = np.min([min_arr,temp_min], axis=0)

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}
csv_feature_dict

In [ ]:
# csv_feature_dict = {'CO2관측치': [0.0, 1757.0],
#  'EC관측치': [-0.00080744, 43.59114074707031],
#  '난방부하': [0.0, 68.50000381469727],
#  '난방온도': [0.0, 26.0],
#  '내부습도관측치': [0.0, 110.6199913],
#  '내부온도관측치': [0.0, 40.59999847412109],
#  '냉방부하': [0.0, 403.5995],
#  '냉방온도': [0.0, 28.0],
#  '레드 LED동작강도': [0.0, 201.0],
#  '백색광추정광량': [0.0, 309.41],
#  '블루 LED동작강도': [0.0, 201.0],
#  '외부습도관측치': [0.0, 201.0],
#  '외부온도관측치': [0.0, 89.4000015258789],
#  '적색광추정광량': [0.0, 165.48],
#  '총추정광량': [0.0, 631.54],
#  '최근분무량': [0.0, 642997.3999999999],
#  '화이트 LED동작강도': [0.0, 201.0]}

In [ ]:
train = pd.DataFrame({'img':[0],'csv':[0],'weight':[0]})
for case in tqdm(train_cases):
  img_train = sorted(glob(case + '/image/*'))
  csv_train = sorted(glob(case + '/meta/*'))
  weights = pd.read_csv(case + '/label.csv')['leaf_weight']
  temp = pd.DataFrame({'img':img_train,'csv':csv_train,'weight':weights})
  train=pd.concat([train,temp], axis=0)
train = train[1:].reset_index(drop=True)
train

100%|██████████| 48/48 [00:00<00:00, 189.65it/s]


,img,csv,weight
0,/content/drive/MyDrive/KIST/open/train/CASE01/...,/content/drive/MyDrive/KIST/open/train/CASE01/...,49.193
1,/content/drive/MyDrive/KIST/open/train/CASE01/...,/content/drive/MyDrive/KIST/open/train/CASE01/...,59.764
2,/content/drive/MyDrive/KIST/open/train/CASE01/...,/content/drive/MyDrive/KIST/open/train/CASE01/...,72.209
3,/content/drive/MyDrive/KIST/open/train/CASE01/...,/content/drive/MyDrive/KIST/open/train/CASE01/...,85.737
4,/content/drive/MyDrive/KIST/open/train/CASE01/...,/content/drive/MyDrive/KIST/open/train/CASE01/...,102.537
...,...,...,...
875,/content/drive/MyDrive/KIST/open/train/CASE74/...,/content/drive/MyDrive/KIST/open/train/CASE74/...,211.497
876,/content/drive/MyDrive/KIST/open/train/CASE74/...,/content/drive/MyDrive/KIST/open/train/CASE74/...,214.116
877,/content/drive/MyDrive/KIST/open/train/CASE74/...,/content/drive/MyDrive/KIST/open/train/CASE74/...,214.293
878,/content/drive/MyDrive/KIST/open/train/CASE75/...,/content/drive/MyDrive/KIST/open/train/CASE75/...,210.872


In [ ]:
img_files = sorted(glob('/content/drive/MyDrive/KIST/open/test/image/*'))
metas = sorted(glob('/content/drive/MyDrive/KIST/open/test/meta/*'))
test = pd.DataFrame({'img':img_files, 'csv':metas})
test

,img,csv
0,/content/drive/MyDrive/KIST/open/test/image/00...,/content/drive/MyDrive/KIST/open/test/meta/001...
1,/content/drive/MyDrive/KIST/open/test/image/00...,/content/drive/MyDrive/KIST/open/test/meta/002...
2,/content/drive/MyDrive/KIST/open/test/image/00...,/content/drive/MyDrive/KIST/open/test/meta/003...
3,/content/drive/MyDrive/KIST/open/test/image/00...,/content/drive/MyDrive/KIST/open/test/meta/004...
4,/content/drive/MyDrive/KIST/open/test/image/00...,/content/drive/MyDrive/KIST/open/test/meta/005...
...,...,...
455,/content/drive/MyDrive/KIST/open/test/image/45...,/content/drive/MyDrive/KIST/open/test/meta/456...
456,/content/drive/MyDrive/KIST/open/test/image/45...,/content/drive/MyDrive/KIST/open/test/meta/457...
457,/content/drive/MyDrive/KIST/open/test/image/45...,/content/drive/MyDrive/KIST/open/test/meta/458...
458,/content/drive/MyDrive/KIST/open/test/image/45...,/content/drive/MyDrive/KIST/open/test/meta/459...


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train', transforms=None):
        self.mode = mode
        self.files = files
        self.csv_feature_dict = csv_feature_dict
        self.csv_feature_check = [0]*len(self.files)
        self.csv_features = [None]*len(self.files)
        self.max_len = 24 * 6
        self.transforms = transforms

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        # file = self.files[i]
        # file_name = file.split('/')[-1]
        image_path = self.files['img'][i]
        csv_path = self.files['csv'][i]
        
        # csv
        if self.csv_feature_check[i] == 0:
            df = pd.read_csv(csv_path)
            df = df[self.csv_feature_dict.keys()]
            df = df.fillna(0)
            # MinMax scaling
            for col in df.columns:
                df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
                df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])
            # zero padding
            pad = np.zeros((self.max_len, len(df.columns)))
            length = min(self.max_len, len(df))
            pad[-length:] = df.to_numpy()[-length:]
            # transpose to sequential data
            csv_feature = pad.T
            self.csv_features[i] = csv_feature
            self.csv_feature_check[i] = 1
        else:
            csv_feature = self.csv_features[i]
        
        # image
        img = cv2.imread(image_path)
        if self.transforms is not None:
          img = self.transforms(image=img)['image']
        
        if self.mode == 'train':
            weight = self.files['weight'][i]
            # json_path = f'{file}/{file_name}.json'
            # with open(json_path, 'r') as f:
            #     json_file = json.load(f)
            
            # crop = json_file['annotations']['crop']
            # disease = json_file['annotations']['disease']
            # risk = json_file['annotations']['risk']
            # label = f'{crop}_{disease}_{risk}'
            
            return {
                'img' : img,
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32),
                # 타겟 타입 float로 바꾸기
                'target' : torch.tensor(weight, dtype=torch.float32)
            }
        else:
            return {
                'img' : img,
                'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32)
            }

In [ ]:
device = torch.device("cuda:0")
batch_size = 4
# class_n = len(label_encoder)
learning_rate = 1e-4
embedding_dim = 600
height = 600
width = 600
num_features = len(csv_feature_dict)
max_len = 24
dropout_rate = 0.1
epochs = 15
k_folds = 5
vision_pretrain = True
save_path = '/content/drive/MyDrive/220518/'

In [ ]:
num_features

17

In [ ]:
# train = sorted(glob('/content/train/*'))
# test = sorted(glob('/content/test/*'))

# labelsss = pd.read_csv('/content/drive/MyDrive/LG/train.csv')['label']

In [ ]:
# train_transforms = A.Compose([
#         A.Resize(height=height, width=width, interpolation=cv2.INTER_AREA),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5),
#         A.Rotate(25, interpolation=cv2.INTER_AREA),
#         A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#         ToTensorV2(),
# ])

train_transforms = A.Compose([
        A.Resize(height=height, width=width, interpolation=cv2.INTER_AREA),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
])

In [ ]:
test_transforms = A.Compose([
        A.Resize(height=height, width=width, interpolation=cv2.INTER_AREA),
        A.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

In [ ]:
train_dataset = CustomDataset(train, transforms = train_transforms)
test_dataset = CustomDataset(test, mode = 'test', transforms = test_transforms)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=False)

In [ ]:
next(iter(train_dataset))

{'csv_feature': tensor([[0.4776, 0.4776, 0.4776,  ..., 0.4591, 0.4591, 0.4617],
         [0.5246, 0.5257, 0.5257,  ..., 0.5369, 0.5369, 0.5358],
         [0.7539, 0.7585, 0.7612,  ..., 0.6382, 0.6473, 0.6554],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]),
 'img': tensor([[[1.7865, 1.8037, 1.7865,  ..., 1.9235, 1.9235, 1.9235],
          [1.8037, 1.8037, 1.8208,  ..., 1.9407, 1.9407, 1.9578],
          [1.7865, 1.8037, 1.8037,  ..., 1.9407, 1.9578, 1.9407],
          ...,
          [1.3584, 1.2385, 1.1700,  ..., 2.0092, 2.0092, 1.9920],
          [1.2728, 1.1529, 1.0502,  ..., 2.0263, 2.0263, 2.0092],
          [1.1872, 1.0502, 0.9646,  ..., 2.0092, 2.0092, 2.0092]],
 
         [[1.4832, 1.4832, 1.4657,  ..., 1.6057, 1.5882, 1.5707],
          [1.4832, 1.4832, 1.5182,  ..., 1.6408, 1.6408, 1.6232],
          [1.4657, 1.4832,

In [ ]:
class CNN_Encoder(nn.Module):
    def __init__(self, rate=0.1):
        super(CNN_Encoder, self).__init__()
        self.model = models.convnext_large(pretrained=True)
        # self.model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

In [ ]:
class RNN_Decoder(nn.Module):
    def __init__(self, max_len, embedding_dim, num_features, rate):
        super(RNN_Decoder, self).__init__()
        self.lstm = nn.LSTM(max_len, embedding_dim)
        self.rnn_fc = nn.Linear(num_features*embedding_dim, 100)
        self.final_layer = nn.Linear(100 + 1000, 1) # resnet out_dim + lstm out_dim
        self.dropout = nn.Dropout(rate)

    def forward(self, enc_out, dec_inp):
        hidden, _ = self.lstm(dec_inp)
        hidden = hidden.view(hidden.size(0), -1)
        hidden = self.rnn_fc(hidden)
        concat = torch.cat([enc_out, hidden], dim=1) # enc_out + hidden 
        fc_input = concat
        output = self.dropout((self.final_layer(fc_input)))
        return output

In [ ]:
class CNN2RNN(nn.Module):
    def __init__(self, max_len, embedding_dim, num_features, rate):
        super(CNN2RNN, self).__init__()
        self.cnn = CNN_Encoder(rate)
        self.rnn = RNN_Decoder(max_len, embedding_dim, num_features, rate)
        
    def forward(self, img, seq):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq)
        
        return output

In [ ]:
model = models.convnext_large(pretrained=True)
# model = model.to(device)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)

Downloading: "https://download.pytorch.org/models/convnext_large-ea097f82.pth" to /root/.cache/torch/hub/checkpoints/convnext_large-ea097f82.pth


  0%|          | 0.00/755M [00:00<?, ?B/s]

## 학습

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

criterion = nn.L1Loss()

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
def accuracy_function(real, pred):    
    real = real.cpu().detach().numpy()
    pred = pred.cpu().detach().numpy()
    score = mean_absolute_error(real, pred)
    return score

def train_step(batch_item, training):
    img = batch_item['img'].to(device)
    csv_feature = batch_item['csv_feature'].to(device)
    label = batch_item['target'].to(device)
    new_shape = (len(label), 1)
    label = label.view(new_shape)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img, csv_feature)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        score = accuracy_function(label, output)
        return loss, score
    else:
        model.eval()
        with torch.no_grad():
            output = model(img, csv_feature)
            loss = criterion(output, label)
        score = accuracy_function(label, output)
        return loss, score

In [ ]:
# loss_plot, val_loss_plot = [], []
# metric_plot, val_metric_plot = [], []

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

# Start print
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_dataset)):
  my_file = Path(save_path+'best_model'+str(fold)+'.pt')

  gc.collect()
  loss_plot, val_loss_plot = [], []
  metric_plot, val_metric_plot = [], []
  
  # Print
  print('')
  print(f'FOLD {fold}')
  print('--------------------------------')

  if my_file.is_file():
    continue
  
  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  # Define data loaders for training and testing data in this fold
  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=batch_size, sampler=train_subsampler, num_workers=2)
  val_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=batch_size, sampler=test_subsampler, num_workers=2)
  
  model = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, rate=dropout_rate)
  # model.load_state_dict(torch.load(save_path+'pretrained_model.pt', map_location=device), strict=False)
  model = model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                          lr_lambda=lambda epoch: 0.95 ** epoch,
                                          last_epoch=-1,
                                          verbose=False)

  criterion = nn.L1Loss()
  
  for epoch in range(epochs):
      gc.collect()
      total_loss, total_val_loss = 0, 0
      total_acc, total_val_acc = 0, 0
      
      tqdm_dataset = tqdm(enumerate(train_dataloader))
      training = True
      for batch, batch_item in tqdm_dataset:
          batch_loss, batch_acc = train_step(batch_item, training)
          total_loss += batch_loss
          total_acc += batch_acc
          
          tqdm_dataset.set_postfix({
              'Epoch': epoch + 1,
              'Loss': '{:06f}'.format(batch_loss.item()),
              'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
              'Mean MAE' : '{:06f}'.format(total_acc/(batch+1))
          })
      loss_plot.append(total_loss/(batch+1))
      metric_plot.append(total_acc/(batch+1))
      
      tqdm_dataset = tqdm(enumerate(val_dataloader))
      training = False
      for batch, batch_item in tqdm_dataset:
          batch_loss, batch_acc = train_step(batch_item, training)
          total_val_loss += batch_loss
          total_val_acc += batch_acc
          
          tqdm_dataset.set_postfix({
              'Epoch': epoch + 1,
              'Val Loss': '{:06f}'.format(batch_loss.item()),
              'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
              'Mean Val MAE' : '{:06f}'.format(total_val_acc/(batch+1))
          })
      val_loss_plot.append(total_val_loss/(batch+1))
      val_metric_plot.append(total_val_acc/(batch+1))
      scheduler.step()

      if np.min(val_metric_plot) == val_metric_plot[-1]:
          torch.save(model.state_dict(), save_path+'best_model'+str(fold)+'.pt')
          print('best')

--------------------------------

FOLD 0
--------------------------------


176it [05:03,  1.72s/it, Epoch=1, Loss=10.851015, Mean Loss=52.921295, Mean MAE=52.921283]
44it [01:22,  1.87s/it, Epoch=1, Val Loss=8.254015, Mean Val Loss=31.866991, Mean Val MAE=31.866991]


best


176it [02:40,  1.10it/s, Epoch=2, Loss=4.250408, Mean Loss=26.774670, Mean MAE=26.774669]
44it [00:56,  1.29s/it, Epoch=2, Val Loss=15.314219, Mean Val Loss=17.725294, Mean Val MAE=17.725295]


best


...

176it [02:35,  1.13it/s, Epoch=4, Loss=12.597031, Mean Loss=16.909979, Mean MAE=16.909978]
44it [00:55,  1.27s/it, Epoch=4, Val Loss=6.082180, Mean Val Loss=6.443008, Mean Val MAE=6.443008]


best


176it [02:35,  1.13it/s, Epoch=5, Loss=88.718430, Mean Loss=18.455111, Mean MAE=18.455108]
44it [00:55,  1.27s/it, Epoch=5, Val Loss=4.660989, Mean Val Loss=7.484109, Mean Val MAE=7.484109]
176it [02:34,  1.14it/s, Epoch=6, Loss=6.877861, Mean Loss=12.753349, Mean MAE=12.753349]
44it [00:56,  1.28s/it, Epoch=6, Val Loss=2.834219, Mean Val Loss=8.775299, Mean Val MAE=8.775298]
176it [02:34,  1.14it/s, Epoch=7, Loss=3.138626, Mean Loss=16.358101, Mean MAE=16.358100]
44it [00:56,  1.27s/it, Epoch=7, Val Loss=3.197095, Mean Val Loss=8.075515, Mean Val MAE=8.075515]
176it [02:34,  1.14it/s, Epoch=8, Loss=3.687250, Mean Loss=12.692388, Mean MAE=12.692388]
44it [00:56,  1.27s/it, Epoch=8, Val Loss=3.207759, Mean Val Loss=9.735043, Mean Val MAE=9.735042]
176it [02:34,  1.14it/s, Epoch=9, Loss=12.599501, Mean Loss=12.006765, Mean MAE=12.006765]
44it [00:56,  1.27s/it, Epoch=9, Val Loss=1.327240, Mean Val Loss=6.877240, Mean Val MAE=6.877241]
176it [02:34,  1.14it/s, Epoch=10, Loss=13.942555, Me

best


176it [02:37,  1.12it/s, Epoch=11, Loss=5.131120, Mean Loss=13.867489, Mean MAE=13.867491]
44it [00:56,  1.28s/it, Epoch=11, Val Loss=6.734963, Mean Val Loss=6.963302, Mean Val MAE=6.963303]
176it [02:35,  1.13it/s, Epoch=12, Loss=3.752617, Mean Loss=14.751315, Mean MAE=14.751319]
44it [00:56,  1.28s/it, Epoch=12, Val Loss=5.853676, Mean Val Loss=8.653214, Mean Val MAE=8.653214]
176it [02:34,  1.14it/s, Epoch=13, Loss=2.272274, Mean Loss=12.688107, Mean MAE=12.688107]
44it [00:56,  1.28s/it, Epoch=13, Val Loss=26.675308, Mean Val Loss=8.626461, Mean Val MAE=8.626460]
176it [02:35,  1.13it/s, Epoch=14, Loss=8.554404, Mean Loss=13.448065, Mean MAE=13.448063]
44it [00:56,  1.28s/it, Epoch=14, Val Loss=8.372533, Mean Val Loss=9.158651, Mean Val MAE=9.158651]
176it [02:35,  1.13it/s, Epoch=15, Loss=3.030064, Mean Loss=12.991530, Mean MAE=12.991535]
44it [00:56,  1.28s/it, Epoch=15, Val Loss=14.349081, Mean Val Loss=6.471069, Mean Val MAE=6.471069]


In [ ]:
batch_size = 4

In [ ]:
def predict(dataset, model0, model1, model2, model3, model4):
    model0.eval()
    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset:
      img = batch_item['img'].to(device)
      seq = batch_item['csv_feature'].to(device)
      with torch.no_grad():
          output = 0.2*model0(img, seq) + 0.2*model1(img, seq) + 0.2*model2(img, seq) + 0.2*model3(img, seq) + 0.2*model4(img, seq)
          # output = 0.5*model0(img, seq) + 0.5*model1(img, seq)
      # output = torch.tensor(torch.argmax(output, dim=1), dtype=torch.int32).cpu().numpy()
      results.extend(output.cpu().numpy())
    return results

model0 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, rate=dropout_rate)
model0.load_state_dict(torch.load(save_path+'best_model'+str(0)+'.pt', map_location=device))
model0.to(device)

model1 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, rate=dropout_rate)
model1.load_state_dict(torch.load(save_path+'best_model'+str(1)+'.pt', map_location=device))
model1.to(device)

model2 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, rate=dropout_rate)
model2.load_state_dict(torch.load(save_path+'best_model'+str(2)+'.pt', map_location=device))
model2.to(device)

model3 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, rate=dropout_rate)
model3.load_state_dict(torch.load(save_path+'best_model'+str(3)+'.pt', map_location=device))
model3.to(device)

model4 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, rate=dropout_rate)
model4.load_state_dict(torch.load(save_path+'best_model'+str(4)+'.pt', map_location=device))
model4.to(device)

preds = predict(test_dataloader, model0, model1, model2, model3, model4)

95it [09:46,  6.19s/it]

In [ ]:
len(preds)

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/KIST/open/sample_submission.csv')

In [ ]:
submission['leaf_weight'] = np.array(preds).reshape(1,-1)[0]
submission

In [ ]:
submission[submission['leaf_weight']<0]

In [ ]:
submission.loc[submission['leaf_weight']<0, 'leaf_weight'] = 0

In [ ]:
images = glob('/content/drive/MyDrive/KIST/open/test/image/*')
images.sort()
for i in range(10):
  image = cv2.imread(images[i])
  print(images[i][-13:])
  print('leaf_weight : ' + str(submission['leaf_weight'].loc[i]))
  plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
submission.to_csv('/content/drive/MyDrive/KIST/submission.csv', index=False)